In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!git clone https://github.com/open-mmlab/Amphion.git
!pip3 install --force einops==0.4.1
!pip install pyworld
!pip install librosa
!pip3 install torch
!pip3 install torchvision
!pip install pydub
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [5]:
from Amphion.models.codec.ns3_codec import FACodecEncoder, FACodecDecoder
from huggingface_hub import hf_hub_download
import torch
import librosa
import soundfile as sf
import os
import glob

In [6]:
path =  "/content/drive/My Drive/IOT/AI/임시"
os.makedirs(path, exist_ok=True)

In [7]:
fa_encoder = FACodecEncoder(
    ngf=32,
    up_ratios=[2, 4, 5, 5],
    out_channels=256,
)

fa_decoder = FACodecDecoder(
    in_channels=256,
    upsample_initial_channel=1024,
    ngf=32,
    up_ratios=[5, 5, 4, 2],
    vq_num_q_c=2,
    vq_num_q_p=1,
    vq_num_q_r=3,
    vq_dim=256,
    codebook_dim=8,
    codebook_size_prosody=10,
    codebook_size_content=10,
    codebook_size_residual=10,
    use_gr_x_timbre=True,
    use_gr_residual_f0=True,
    use_gr_residual_phone=True,
)

encoder_ckpt = hf_hub_download(repo_id="amphion/naturalspeech3_facodec", filename="ns3_facodec_encoder.bin")
decoder_ckpt = hf_hub_download(repo_id="amphion/naturalspeech3_facodec", filename="ns3_facodec_decoder.bin")

fa_encoder.load_state_dict(torch.load(encoder_ckpt))
fa_decoder.load_state_dict(torch.load(decoder_ckpt))

fa_encoder.eval()
fa_decoder.eval()

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


ns3_facodec_encoder.bin:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

ns3_facodec_decoder.bin:   0%|          | 0.00/398M [00:00<?, ?B/s]

FACodecDecoder(
  (quantizer): ModuleList(
    (0): ResidualVQ(
      (layers): ModuleList(
        (0): FactorizedVectorQuantize(
          (in_proj): Linear(in_features=256, out_features=8, bias=True)
          (out_proj): Linear(in_features=8, out_features=256, bias=True)
          (_codebook): Embedding(1024, 8)
        )
      )
    )
    (1): ResidualVQ(
      (layers): ModuleList(
        (0-1): 2 x FactorizedVectorQuantize(
          (in_proj): Linear(in_features=256, out_features=8, bias=True)
          (out_proj): Linear(in_features=8, out_features=256, bias=True)
          (_codebook): Embedding(1024, 8)
        )
      )
    )
    (2): ResidualVQ(
      (layers): ModuleList(
        (0-2): 3 x FactorizedVectorQuantize(
          (in_proj): Linear(in_features=256, out_features=8, bias=True)
          (out_proj): Linear(in_features=8, out_features=256, bias=True)
          (_codebook): Embedding(1024, 8)
        )
      )
    )
  )
  (model): Sequential(
    (0): Conv1d(256, 

In [12]:
input_folder = '/content/drive/MyDrive/IOT/Human/drama/wav'
output_folder = '/content/drive/MyDrive/IOT/AI/임시'
os.makedirs(output_folder, exist_ok=True)

# 폴더 내 모든 .wav 파일에 대해 처리
wav_files = glob(os.path.join(input_folder, '*.wav'))

In [13]:
len(wav_files)

80

In [ ]:
import os
import glob

for wav_path in wav_files:
    print(f"Processing {wav_path}")
    # 음성 파일 로드
    test_wav, sr = librosa.load(wav_path, sr=16000)
    test_wav = torch.from_numpy(test_wav).float()
    test_wav = test_wav.unsqueeze(0).unsqueeze(0)

    with torch.no_grad():
        # encode
        enc_out = fa_encoder(test_wav)

        # quantize
        vq_post_emb, vq_id, _, quantized, spk_embs = fa_decoder(enc_out, eval_vq=False, vq=True)

        # decode (recommand)
        recon_wav = fa_decoder.inference(vq_post_emb, spk_embs)

    # 변환된 파일 저장
    output_path = os.path.join(output_folder, os.path.basename(wav_path))
    sf.write(output_path, recon_wav[0][0].cpu().numpy(), 16000, format='wav', subtype='PCM_16')
    print(f"Saved processed file to {output_path}")

Processing /content/drive/MyDrive/IOT/Human/drama/wav/너 나이가 몇인데 아직도 엄마가 니 방청소를 해야 돼!.wav
